# TMA Capacitor Bank discharge vs. current draw profiles

# SITCOM-1224

**title**: TMA (Telescope Mount Assembly) Capacitor Bank discharge vs. current draw profiles

**Description**: Continue in the analysis started in SITCOM-1146 (https://jira.lsstcorp.org/browse/SITCOM-1146). 
This ticket for looking at the relationship between the current draw (rather than power supply voltage) 
and also how much power was in the banks prior to a large drop. This was explored a bit in SITCOM-1146, but could be expanded.

**Author**: Laura Toribio San Cipriano

2024-03-13

### Setup

In [57]:
%matplotlib inline
%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from lsst_efd_client import EfdClient, resample
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState

from astropy.time import Time, TimeDelta
from astropy.table import Table
from scipy.interpolate import UnivariateSpline

In [59]:
client = EfdClient('usdf_efd')

In [60]:
tma_power_topic = 'lsst.sal.MTMount.mainPowerSupply'
tma_el_topic = 'lsst.sal.MTMount.elevation'
tma_az_topic = 'lsst.sal.MTMount.azimuth'

### Get Data

In [61]:
obs_days = [20240301]#, 20240302, 20240303, 20240304, 20240305]

In [62]:
def get_events(dayObs):
    """
    Identify slew events for a given dayObs
    """
    eventMaker = TMAEventMaker()
    events = eventMaker.getEvents(dayObs)
    
    # Get lists of slew and track events
    slews = [e for e in events if e.type == TMAState.SLEWING]
    tracks = [e for e in events if e.type == TMAState.TRACKING]
    print(f"{dayObs}: Found {len(slews)} slews and {len(tracks)} tracks")
    return slews

In [63]:
def get_acc_jerk(time, velocity):
    """
    From velocity data, calculate acceleration and jerk
    """
    try:
       smoothing_factor=0.2
       kernel_size = 5
       kernel = np.ones(kernel_size) / kernel_size
    
       VelSpline = UnivariateSpline(time, velocity, s=0)
       smoothedVel = np.convolve(VelSpline(time), kernel, mode='same')
       VelSpline = UnivariateSpline(time, smoothedVel, s=smoothing_factor)
    
       AccSpline = VelSpline.derivative(n=1)
       smoothedAcc = np.convolve(AccSpline(time), kernel, mode='same')
       AccSpline = UnivariateSpline(time, smoothedAcc, s=smoothing_factor)
   
       JerkSpline = AccSpline.derivative(n=1) 

       return VelSpline(time), AccSpline(time), JerkSpline(time)
    except:
        return False, False, False

In [65]:
async def data_per_slew(i, slew, debug=False):
    """
    get TMA velocity data, power data for a given slew and e currren draw

    Calculate maximum velocity, acceleration and jerk values and minimum power draw
    """
    data = [i, slew.dayObs, slew.duration, slew.begin, slew.end]

    power_df = await client.select_time_series(tma_power_topic, '*', slew.begin, slew.end)
    data.append(np.min(power_df.powerSupplyVoltage))
    data.append(np.max(power_df.powerSupplyCurrent))
    
    az = await client.select_time_series(tma_az_topic, ['*'], slew.begin, slew.end)
    el = await client.select_time_series(tma_el_topic, ['*'], slew.begin, slew.end)
   
    AzBegin = az.actualPosition.iloc[0]
    AzEnd   = az.actualPosition.iloc[-1]
    ElBegin = el.actualPosition.iloc[0]
    ElEnd   = el.actualPosition.iloc[-1]
    
    AzVel, AzAcc, AzJerk = get_acc_jerk(az.timestamp, az.actualVelocity)
    ElVel, ElAcc, ElJerk = get_acc_jerk(el.timestamp, el.actualVelocity)

    for var in [AzVel, AzAcc, AzJerk, ElVel, ElAcc, ElJerk, AzBegin, AzEnd, ElBegin, ElEnd]:
        data.append(np.max(np.abs(var)))
    if debug:
        return az, el, power_df, data
    else:
        return data

In [66]:
Data = []
failed = []
data_names = ['slew_id','dayobs','duration','begin','end','min_power','max_current','az_max_vel','az_max_acc','az_max_jerk','el_max_vel','el_max_acc','el_max_jerk', 'az_pos_begin', 'az_pos_end', 'el_pos_begin', 'el_pos_end'] #, 'dis_slew'
for dayObs in obs_days:
    try:
        slews = get_events(dayObs)
    except:
        print(dayObs)
    for i, slew in enumerate(slews):
        try:
            d = await data_per_slew(i, slew)
            Data.append(d)
        except Exception as e:
            failed.append(i)
            #print(i, e)
df = pd.DataFrame(np.vstack(Data),columns=data_names)


20240301: Found 375 slews and 350 tracks


In [ ]:
#Remove extreme outliers
df = df[(df.el_max_jerk < 100)&(df.duration<3000)]

### Analysis

In [ ]:
#### Relationship of min power supply voltage and distance of slew